## CSV 2024

### 1. 모듈 불러오기

In [40]:
# 디렉토리 경로
import os

# 데이터셋 관련
from detectron2 import model_zoo
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog

# 학습 관련
import copy
import torch

from detectron2.config import get_cfg
import detectron2.data.transforms as T

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_train_loader
from detectron2.data import detection_utils as utils

# 기타
import wandb
from detectron2.utils.logger import setup_logger

setup_logger()


<Logger detectron2 (DEBUG)>

### 2. 데이터 등록

##### 리소스 등록 및 경로 설정(default)

In [41]:
# 리소스
coco_dataset_train = 'coco_trash_train'
coco_dataset_test = 'coco_trash_test'

# 경로
path_dataset = '/data/ephemeral/home/dataset/'
path_output_eval = './output_eval'

##### 데이터셋 등록

In [42]:
if coco_dataset_train not in DatasetCatalog.list():
    register_coco_instances(coco_dataset_train, {}, path_dataset + 'train.json', path_dataset)

if coco_dataset_test not in DatasetCatalog.list():
    register_coco_instances(coco_dataset_test, {}, path_dataset + 'test.json', path_dataset)

data_size = len(DatasetCatalog.get(coco_dataset_train))
MetadataCatalog.get(coco_dataset_train).thing_classes=['General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 
                                                       'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']

[10/12 12:54:25 d2.data.datasets.coco]: Loading /data/ephemeral/home/dataset/train.json takes 15.30 seconds.
[10/12 12:54:25 d2.data.datasets.coco]: Loaded 4883 images in COCO format from /data/ephemeral/home/dataset/train.json


In [43]:
wandb.init(project="2024 부스트캠프 재활용품 분류대회(22, CSV)",)

#### *수정이 필요한 리소스

In [44]:
path_model_pretrained = 'COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'
backbone = 'build_resnet_fpn_backbone'

### 3. 초기 설정

#### 기본 설정

In [45]:
cfg = get_cfg()

# COCO 사전 학습 가중치 경로 설정
cfg.merge_from_file(model_zoo.get_config_file(path_model_pretrained))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(path_model_pretrained)

cfg.DATASETS.TRAIN = (coco_dataset_train,)
cfg.DATASETS.TEST = (coco_dataset_test,)

cfg.SEED = 2024
cfg.DATALOADER.NUM_WOREKRS = 4

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10                     # 분류할 클래스 수
cfg.TEST.EVAL_PERIOD = 3000                              # 검증 주기(단위 : iter)

schedulers = ['WarmupMultiStepLR',]

#### *하이퍼파라미터

In [46]:
cfg.SOLVER.IMS_PER_BATCH = 4                              # 배치크기
cfg.SOLVER.BASE_LR = 0.001                                # 초기 학습률
cfg.SOLVER.MAX_ITER = 15000                               # 최대 학습 반복 수
cfg.SOLVER.STEPS = (8000, 12000)                          # 학습률 감소 단계
cfg.SOLVER.GAMMA = 0.1                                    # 학습률 감소 비율(%)

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128            # 각 이미지당 샘플링할 RoI 개수
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.4
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7 

cfg.MODEL.RPN.NMS_THRESH = 0.6

cfg.MODEL.BACKBONE.NAME = backbone
cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]
# cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[32, 64, 128, 256, 512]]
# cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.5, 1.0, 2.0]]
# cfg.MODEL.ANCHOR_GENERATOR.ANGLES = [[-90, 0, 90]]
cfg.SOLVER.LR_SCHEDULER_NAME = schedulers[0]
print(cfg.dump())


aug_list = [                                            # 데이터 증강 옵션
    T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
    T.RandomBrightness(0.8, 1.8),
    T.RandomContrast(0.6, 1.3),
    T.RandomRotation(angle=[-15, 15]),
    T.RandomCrop(crop_type="relative_range", crop_size=(0.8, 0.8)),
    T.RandomLighting(0.1)
]

config = {
    "model": path_model_pretrained.split('/')[1],
    "backbone": backbone,
    "roi size": cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE,
    "epochs": (cfg.SOLVER.IMS_PER_BATCH * cfg.SOLVER.MAX_ITER) // data_size,
    "learning rate": cfg.SOLVER.BASE_LR,
    "scheduler": cfg.SOLVER.LR_SCHEDULER_NAME,
    "augmentation": aug_list,
}
wandb.config.update(config)

CUDNN_BENCHMARK: false
DATALOADER:
  ASPECT_RATIO_GROUPING: true
  FILTER_EMPTY_ANNOTATIONS: true
  NUM_WOREKRS: 4
  NUM_WORKERS: 4
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: []
  PROPOSAL_FILES_TRAIN: []
  TEST:
  - coco_trash_test
  TRAIN:
  - coco_trash_train
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: false
    SIZE:
    - 0.9
    - 0.9
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN:
  - 640
  - 672
  - 704
  - 736
  - 768
  - 800
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
MODEL:
  ANCHOR_GENERATOR:
    ANGLES:
    - - -90
      - 0
      - 90
    ASPECT_RATIOS:
    - - 0.5
      - 1.0
      - 2.0
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES:
    - - 32
    - - 64
    - - 128
    - - 256
    - - 512
  BACKBONE:
    FREEZE_AT: 2

### 4. 학습

#### Augmentation, 전처리

In [47]:
def MyMapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict['file_name'], format='BGR')
    
    transform_list = aug_list

    image, transforms = T.apply_transform_gens(transform_list, image)
    
    dataset_dict['image'] = torch.as_tensor(image.transpose(2,0,1).astype('float32'))
    
    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop('annotations')
        if obj.get('iscrowd', 0) == 0
    ]
    
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict['instances'] = utils.filter_empty_instances(instances)
    
    return dataset_dict

In [48]:
class MyTrainer(DefaultTrainer):
    
    @classmethod
    def build_train_loader(cls, cfg, sampler=None):
        return build_detection_train_loader(
            cfg, mapper=MyMapper, sampler=sampler
        )


    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs(path_output_eval, exist_ok=True)
            output_folder = path_output_eval
        
        return COCOEvaluator(dataset_name, output_dir=output_folder)
    

    # # Wandb 기록용
    # def log_metrics(self, results):
    #     wandb.log({
    #         "bbox_AP": results["bbox"]["AP"],
    #         "bbox_AP50": results["bbox"]["AP50"],
    #         "bbox_AP75": results["bbox"]["AP75"],
    #         "bbox_IoU": results["bbox"]["AP"],
    #     })


    # # Wandb 기록용
    # def evaluate_model(self):
    #     evaluator = COCOEvaluator(coco_dataset_test, self.cfg, False, output_dir=path_output_eval)
    #     val_loader = build_detection_test_loader(self.cfg, coco_dataset_test)
    #     results = inference_on_dataset(self.build_model(self.cfg), val_loader, evaluator)
    #     self.log_metrics(results)

In [49]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok = True)

trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)

trainer.train()
# trainer.evaluate_model()

wandb.finish

[10/12 12:54:30 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (11, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (11,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (40, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (40,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias

[10/12 12:54:30 d2.engine.train_loop]: Starting training from iteration 0
[10/12 12:54:43 d2.utils.events]:  eta: 2:23:54  iter: 19  total_loss: 3.022  loss_cls: 2.461  loss_box_reg: 0.4735  loss_rpn_cls: 0.04056  loss_rpn_loc: 0.03537    time: 0.5871  last_time: 0.6359  data_time: 0.0309  last_data_time: 0.0127   lr: 1.9981e-05  max_mem: 14984M
[10/12 12:54:55 d2.utils.events]:  eta: 2:29:54  iter: 39  total_loss: 2.526  loss_cls: 1.886  loss_box_reg: 0.4176  loss_rpn_cls: 0.06398  loss_rpn_loc: 0.04087    time: 0.5947  last_time: 0.6106  data_time: 0.0146  last_data_time: 0.0142   lr: 3.9961e-05  max_mem: 15208M
[10/12 12:55:06 d2.utils.events]:  eta: 2:29:42  iter: 59  total_loss: 1.559  loss_cls: 0.9693  loss_box_reg: 0.467  loss_rpn_cls: 0.02257  loss_rpn_loc: 0.02615    time: 0.5927  last_time: 0.5538  data_time: 0.0137  last_data_time: 0.0131   lr: 5.9941e-05  max_mem: 15208M
[10/12 12:55:18 d2.utils.events]:  eta: 2:30:25  iter: 79  total_loss: 1.14  loss_cls: 0.5719  loss_box_

<function wandb.sdk.wandb_run.finish(exit_code: Optional[int] = None, quiet: Optional[bool] = None) -> None>